### Imitando o estilo de escrita de um autor(a) com rede LSTM

In [ ]:
from IPython.display import Image #Para visualizar imagens explicativas
Image(filename='Alice.jpg') 

Será que conseguimos imitar o estilo de escrita de um autor(a)? 

Vamos aprender como uma rede LSTM pode nos ajudar a prever palavras com base em últimas sequências.

In [ ]:
import tensorflow as tf, numpy as np 
from tensorflow import keras 
from keras import layers 
from keras.preprocessing.text import Tokenizer 
from tensorflow.keras.utils import pad_sequences
from keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.utils import to_categorical

import matplotlib.pyplot as plt
import re, string # expressões regulares
plt.style.use('ggplot')

Iniciaremos lendo o arquivo inteiro em uma unica string para removermos pontuação. 

Feito isso adicionamos um espaço após cada quebra de linha “\n” para que a divisão das palavras em text.split() considere a quebra de linha como uma palavra (Fazemos isso para que a rede use essa palavra “quebra de linha” quando decidir iniciar uma nova linha)

In [ ]:
text = open('wonderland.txt', errors="ignore").read() #lendo o arquivo

In [ ]:
text

In [ ]:
print(text[:100])

In [ ]:
text = text.lower()                                   # somente letras minusculas
text = text.replace('\n', ' \n ')                     # considerar \n (espaço) como palavra
text_words = [w for w in text.split(' ') if w.strip() != '' or w == '\n']
text_words = [re.sub(r'[^\x00-\x7f]',r'', s) for s in text_words] # remove numeros e acentos
print('Quantidade Total de Palavras: ', len(text_words))
print(text_words[:50])

### Criando um dicionário de palavras

Tendo agora as palavras em uma **grande lista**, usamos a classe **Tokenizer** para tokenização das palavras e criação de um vocabulário com as 500 palavras mais frequentes no texto(parâmetro num_words do Tokenizer):

**Tokenizar** é a tarefa de cortar um texto em pedaços chamados tokens e, ao mesmo tempo, jogar fora alguns caracteres não úteis, como por exemplo pontuação. 

In [ ]:
vocab_size = 200 # tamanho do vocabulário

# criando tokenizer (usa somente as 'VOCAB_SIZE' palavras mais comuns)
tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>', #<OOV> palavras que não estão no dicionário
                      filters='[^\x00-\x7f]')

# definindo vocabulario
tokenizer.fit_on_texts(text_words)

# tokenizando as palavras
tokens = tokenizer.texts_to_sequences(text_words)
tokens = [int(t[0]) for t in tokens] # lista para inteiros
print(text_words[:10])
print(tokens[:10])

In [ ]:
text_words[7]

In [ ]:
tokens[7]

### Definindo no alvo:

Após tokenizar as sentenças, devemos agrupar as palavras para criar as sub-sequencias de entrada(X) e as respectivas saídas, ou seja, as palavras seguintes(Y). 

Ao fatiar a lista podemos definir um intervalo entre cada sequência (step no código abaixo), isso reduz a produção de sequencias com palavras sobrepostas e repetidas:

In [ ]:
step = 1 # distancia a cada fatia 
sentences = [] # frases X
next_words = [] # palavras Y
seq_len = 10 # número de tokens por frase
for i in range(0, len(tokens) - seq_len, step):
    # Only add sequences where no word is in ignored_words
    sentences.append(tokens[i: i + seq_len])
    next_words.append(tokens[i + seq_len])
   
print(f'[+] Instâncias para treino: {len(sentences)}')
for i in range(5):
    print(sentences[i],'-->',next_words[i])

In [ ]:
text_words[24]

O ultimo passo de preprocessamento é codificar cada token usando a estratégia one-hot. Após a codificação cada token será representado por um vetor de 500 dimensões (tamanho do vocabulário).

Ao utilizar vetorização one-hot temos que cada palavra é representada por um vetor do tamanho do vocabulário onde somente uma posição é = 1 (“hot”). Como vamos utilizar redes recorrentes com camadas LSTM, usaremos uma sequência de palavras como entrada, ao invés de uma única palavra. 

A camada LSTM itera sobre a sequência de palavras produzindo saídas, porém somente a ultima saída é usada para alimentar as ultimas camadas (return_sequences=False). Em seguida temos uma camada densa que aplica a função.

In [ ]:
Image(filename='OneHot.png') 

In [ ]:
# mostra shape atual
train_x = np.array(sentences) # pega as sentenças
train_y = np.array(next_words) # pega as palavras seguintes
print('Shape X:',train_x.shape)
print('Shape Y:',train_y.shape)

# codificando para one-hot
train_x_onehot = to_categorical(train_x, num_classes=vocab_size)
train_y_onehot = to_categorical(train_y, num_classes=vocab_size)

# train_x:(num_exemplos, num_tokens, vocab_size)
print('Shape X após one-hot:',train_x_onehot.shape)

# train_y:(num_exemplos,vocab_size)
print('Shape Y após one-hot:',train_y_onehot.shape)

In [ ]:
train_x[1]

### Definição e Treino da Rede Recorrente LSTM

In [ ]:
seq_len

In [ ]:
model = keras.Sequential([
    layers.LSTM(64, input_shape=(seq_len, vocab_size), return_sequences=False,), #entrada dos dados
    layers.Dense(vocab_size, activation='softmax') #Cada unidade representa a probabilidade de cada palavra do vocabulário ser a próxima com base na sequencia de entrada.
])

Como a saída do modelo é uma lista com a distribuição de probabilidade de uma classificação multiclasse , é adequado utilizar a função de custo categorical_crossentropy:

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
train_history = model.fit(train_x_onehot, train_y_onehot, validation_split=0.2,
                    epochs=40, batch_size=64)

## Analisando a curva de aprendizado

In [ ]:
plt.plot(train_history.history['accuracy'], label='Treino')
plt.plot(train_history.history['val_accuracy'], label='Validação')
plt.xlabel('Epocas');plt.ylabel('Acurácia')
plt.legend()
plt.show()

Com certeza poderiamos aumentar as epocas de processamento e obter cada vez uma acurácia maior e um erro menor.

### Escrevendo como  Lewis Carroll

Após termos o modelo treinado podemos utiliza-lo para prever algumas palavras.

Começaremos com a palavra seed “Alice:”. 

Precisamos que a sequencia de entrada tenha o tamanho seq_len definido, no nosso caso 10 palavras. Para isso utilizamos o auxiliar pad_sequences que insere espaços à esquerda da sequencia de forma a preencher as posições faltantes:

In [ ]:
seed_text = ['Alice:'] # frase inicial
seed_tokens = tokenizer.texts_to_sequences(seed_text)[0] # substitui palavras por tokens

# preenche sequencia com zeros para ter o comprimento adequado pra rede
tokens_x = pad_sequences([seed_tokens], maxlen=seq_len, ) #10

tokens_x = to_categorical(tokens_x, num_classes=vocab_size) # one hot
pred_y = model.predict(tokens_x)[0] # preve probabilidades para a proxima palavra

print(f'Quantidade de Probabilidades: {len(pred_y)}')
print(pred_y[:10])

Tendo as probabilidades de cada palavra em mãos podemos selecionar a palavra com maior probabilidade ou pegar uma outra palavra que possui uma das mais altas probabilidade. Como cada valor de pred_y é a probabilidade para cada índice do token, realizamos a conversão desse indice para a palavra, segundo o nosso dicionário:

In [ ]:
# pega indice da palavra com maior probabilidade 
next_token = np.argmax(pred_y,)

# realiza a inversão de token para palavra
next_word = tokenizer.sequences_to_texts([[next_token]])
print('Proximo token: ', next_token, '-->', next_word)

Uma maneira mais interessante de escolher a próxima palavra é usar uma distribuição categórica para calcular o índice do próximo token. Para isso definiremos uma função sample_word:

In [ ]:
def sample_word(pred_y, temperature=1.0):
    pred_y = pred_y / temperature # 'força' das probabilidades
    pred_token = tf.random.categorical(pred_y, 1).numpy()
    return pred_token # token de saída

Essa função toma como entrada a distribuição de probabilidades das palavras (saída da rede) e amostra um dos tokens com base nesses valores. Note que mesmo com a mesma entrada pred_y essa função retornará diferentes valores, devido a sua natureza probabilistica. Vamos testar:

In [ ]:
seed_text = ['Alice:'] # frase inicial
seed_tokens = tokenizer.texts_to_sequences(seed_text)[0] # substitui palavras por tokens
print(seed_text, ' tokenizado fica: ', seed_tokens)

# preenche sequencia com zeros para ter o comprimento adequado pra rede
tokens_x = pad_sequences([seed_tokens], maxlen=seq_len, )

tokens_x = to_categorical(tokens_x, num_classes=vocab_size) # one hot
pred_y = model.predict(tokens_x) # preve probabilidades para a proxima palavra
next_token = sample_word(pred_y)

# realiza a inversão de token para palavra
next_word = tokenizer.sequences_to_texts(next_token)
print('Proximo token: ', next_token, '-->', next_word)

Para gerar um texto completo, simplesmente colocamos o código acima dentro de um loop, anexando a palavra de saída à frase original, lembrando sempre de aplicar as mesmas etapas de pré-processamento realizadas nas frases do treino:

In [ ]:
seed_text = ['Alice:'] # frase inicial
next_words = 50 # 50 próximas palavras

# substitui palavras por tokens
seed_tokens = tokenizer.texts_to_sequences(seed_text)[0] # substitui palavras por tokens
print(seed_text, ' tokenizado fica: ', seed_tokens)

for _ in range(next_words):
    # preenche sequencia com zeros para ter o comprimento adequado pra rede
    tokens_x = pad_sequences([seed_tokens], maxlen=seq_len, )
    # transforma tokens em vetor one-hot
    tokens_x = to_categorical(tokens_x, num_classes=vocab_size) # one hot
    # preve probabilidades para a proxima palavra
    pred_y = model.predict(tokens_x)
    # faz amostragem com base nas probabilidades
    next_token = sample_word(pred_y, 0.2)
    next_token = next_token.flatten()[0] # pega valor como um int
    # anexa token a lista
    seed_tokens.append(next_token) 
    
# como a saída é um conjunto de tokens
# realiza a inversão para palavras, usando word_index
resultado = tokenizer.sequences_to_texts([seed_tokens])
print('\n')
print(resultado[0])